in this notebook we:
* take the model from stage 4a, and test it on rescaled images

In [1]:
from pathlib import Path
import json
import logging

import numpy as np
import pandas as pd

from fastai.vision import *

import os

import wandb
from wandb.fastai import WandbCallback

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import configparser

# from functions_wandb_no_rescale import *
from functions_wandb import *

In [2]:
%matplotlib inline

In [20]:
df = pd.read_csv(PATH_TO_TRAIN_DF / "train_valid_df.csv")

classes = df.species.unique()
print(len(classes)) # no "Mammal_Other"
classes

28


array(['Human', 'Blank', 'Elephant_African', 'Hog_Red_River', 'Buffalo_African', 'Leopard_African', 'Monkey',
       'Duiker_Red', 'Civet_African_Palm', 'Squirrel', 'Duiker_Blue', 'Bird', 'Mongoose_Black_Footed', 'Rodent',
       'Duiker_Yellow_Backed', 'Genet', 'Chimpanzee', 'Gorilla', 'Mongoose', 'Porcupine_Brush_Tailed', 'Pangolin',
       'Mandrillus', 'Chevrotain_Water', 'Cat_Golden', 'Rat_Giant', 'Guineafowl_Crested', 'Guineafowl_Black',
       'Rail_Nkulengu'], dtype=object)

In [21]:
len(df.uniqueName)

347120

In [22]:
df.head()

,uniqueName,species,main_folder,is_valid,sub_folder
0,0000005.jpg,Human,Compressed Camera Trap Images,False,T33
1,0000007.jpg,Human,Compressed Camera Trap Images,False,T33
2,0000009.jpg,Human,Compressed Camera Trap Images,False,T33
3,0000013.jpg,Human,Compressed Camera Trap Images,False,T33
4,0000015.jpg,Human,Compressed Camera Trap Images,False,T33


In [23]:
df["uniqueName"] = df.uniqueName.apply(lambda x: "/data_rescaled/resc_" + x)
df.head()

,uniqueName,species,main_folder,is_valid,sub_folder
0,/data_rescaled/resc_0000005.jpg,Human,Compressed Camera Trap Images,False,T33
1,/data_rescaled/resc_0000007.jpg,Human,Compressed Camera Trap Images,False,T33
2,/data_rescaled/resc_0000009.jpg,Human,Compressed Camera Trap Images,False,T33
3,/data_rescaled/resc_0000013.jpg,Human,Compressed Camera Trap Images,False,T33
4,/data_rescaled/resc_0000015.jpg,Human,Compressed Camera Trap Images,False,T33


In [24]:
df = df[df.is_valid]
len(df)

43628

In [25]:
model_folder = "saved_models"
model = "stage4a-intermediate_bestmodel.pkl"

In [26]:
learn = load_learner(model_folder, model, test=df.uniqueName)

In [38]:
preds, y = learn.get_preds(ds_type=DatasetType.Test)

In [53]:
preds = pd.DataFrame(
        np.stack(preds),
        columns=learn.data.classes)

ValueError: Shape of passed values is (28, 1), indices imply (28, 28)

In [54]:
preds

,Bird,Blank,Buffalo_African,Cat_Golden,Chevrotain_Water,Chimpanzee,Civet_African_Palm,Duiker_Blue,Duiker_Red,Duiker_Yellow_Backed,...,Mandrillus,Mongoose,Mongoose_Black_Footed,Monkey,Pangolin,Porcupine_Brush_Tailed,Rail_Nkulengu,Rat_Giant,Rodent,Squirrel
0,5.743499e-06,0.137567,3.323529e-05,1.120363e-06,0.000003,4.746595e-07,1.822831e-07,3.081188e-06,6.339939e-06,1.000803e-04,...,2.286695e-07,6.076862e-07,6.287330e-07,1.863103e-06,4.588552e-07,1.408720e-06,1.099656e-05,1.146514e-06,7.028041e-06,1.268864e-06
1,5.741766e-07,0.000460,5.746752e-05,1.066961e-06,0.000002,1.512224e-05,1.151561e-06,1.506496e-07,1.030288e-05,2.103420e-04,...,1.316583e-07,1.250922e-07,1.789757e-06,7.188823e-06,1.145261e-06,8.504273e-06,2.844499e-05,4.867821e-06,1.162655e-06,3.473887e-07
2,2.403675e-07,0.000269,2.807189e-05,4.353024e-07,0.000001,4.392753e-06,1.155500e-06,2.276430e-08,7.413633e-07,3.165718e-05,...,5.986816e-08,2.603645e-08,8.714592e-07,8.929987e-07,2.496853e-07,3.889070e-06,9.258204e-06,3.537069e-06,9.993730e-07,1.498897e-07
3,9.204893e-06,0.002974,4.408555e-04,1.381217e-06,0.000003,1.958480e-04,1.613991e-06,1.086270e-05,1.205530e-04,5.158943e-02,...,3.707867e-07,1.185252e-05,7.877673e-06,3.657204e-05,4.897071e-06,4.277200e-06,2.462577e-05,3.430950e-07,4.934501e-06,2.958342e-06
4,3.429205e-07,0.992329,5.335137e-05,1.786373e-06,0.000001,9.202054e-08,7.351677e-07,3.037910e-07,6.441360e-07,1.755551e-06,...,3.345131e-06,6.243525e-07,2.713971e-06,1.508487e-05,3.541542e-06,3.289010e-06,1.894127e-06,6.333711e-07,1.163913e-05,9.687103e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43623,1.663137e-03,0.001725,3.458983e-05,1.380556e-04,0.000029,1.352274e-03,6.608617e-05,5.736975e-03,1.284553e-02,1.925576e-04,...,2.293554e-04,2.029100e-04,1.417332e-05,2.187160e-03,5.722069e-05,3.248806e-05,3.190487e-05,4.596190e-05,2.554128e-05,1.658706e-03
43624,4.539652e-05,0.000130,9.310187e-07,6.942720e-06,0.000002,4.915390e-04,3.323348e-06,7.082932e-07,9.804475e-06,2.017840e-05,...,8.790900e-07,4.718187e-07,4.054961e-07,4.474161e-06,1.293553e-05,5.689401e-07,1.897237e-05,1.782077e-07,1.560462e-06,2.459209e-07
43625,1.762118e-04,0.004880,5.249129e-06,1.339516e-04,0.000060,5.211334e-01,5.145297e-05,4.796041e-04,7.022453e-05,4.422554e-06,...,1.710422e-02,7.146603e-05,1.681051e-05,3.075451e-03,6.149261e-06,8.358287e-05,4.142888e-05,3.136026e-05,9.774395e-06,4.366212e-04
43626,3.938063e-06,0.000032,4.535523e-08,5.333808e-07,0.000001,4.047143e-07,6.454020e-07,4.682421e-06,6.077072e-06,3.763003e-07,...,4.775329e-06,1.722424e-07,7.660564e-08,1.609535e-07,1.787171e-07,1.906631e-06,5.094130e-07,2.151043e-06,3.929028e-07,9.069347e-06


In [57]:
sort_classes = sorted(classes)
df_preds = preds.copy()
ranks = df_preds.rank(axis=1,method='dense', ascending=False).astype(int)

df_preds["pred_1"] = pd.Series(ranks.where(ranks==1).notnull().values.nonzero()[1]).apply(lambda x: sort_classes[x])
df_preds["pred_2"] = pd.Series(ranks.where(ranks==2).notnull().values.nonzero()[1]).apply(lambda x: sort_classes[x])
df_preds["pred_3"] = pd.Series(ranks.where(ranks==3).notnull().values.nonzero()[1]).apply(lambda x: sort_classes[x])

df_preds["score_1"] = df_preds.apply(lambda x: x[x.pred_1], axis=1)
df_preds["score_2"] = df_preds.apply(lambda x: x[x.pred_2], axis=1)
df_preds["score_3"] = df_preds.apply(lambda x: x[x.pred_3], axis=1)


In [58]:
df_preds

,Bird,Blank,Buffalo_African,Cat_Golden,Chevrotain_Water,Chimpanzee,Civet_African_Palm,Duiker_Blue,Duiker_Red,Duiker_Yellow_Backed,...,Rail_Nkulengu,Rat_Giant,Rodent,Squirrel,pred_1,pred_2,pred_3,score_1,score_2,score_3
0,5.743499e-06,0.137567,3.323529e-05,1.120363e-06,0.000003,4.746595e-07,1.822831e-07,3.081188e-06,6.339939e-06,1.000803e-04,...,1.099656e-05,1.146514e-06,7.028041e-06,1.268864e-06,Human,Blank,Elephant_African,0.859337,0.137567,0.002829
1,5.741766e-07,0.000460,5.746752e-05,1.066961e-06,0.000002,1.512224e-05,1.151561e-06,1.506496e-07,1.030288e-05,2.103420e-04,...,2.844499e-05,4.867821e-06,1.162655e-06,3.473887e-07,Human,Elephant_African,Blank,0.998418,0.000642,0.000460
2,2.403675e-07,0.000269,2.807189e-05,4.353024e-07,0.000001,4.392753e-06,1.155500e-06,2.276430e-08,7.413633e-07,3.165718e-05,...,9.258204e-06,3.537069e-06,9.993730e-07,1.498897e-07,Human,Blank,Elephant_African,0.999540,0.000269,0.000089
3,9.204893e-06,0.002974,4.408555e-04,1.381217e-06,0.000003,1.958480e-04,1.613991e-06,1.086270e-05,1.205530e-04,5.158943e-02,...,2.462577e-05,3.430950e-07,4.934501e-06,2.958342e-06,Human,Duiker_Yellow_Backed,Elephant_African,0.939456,0.051589,0.004940
4,3.429205e-07,0.992329,5.335137e-05,1.786373e-06,0.000001,9.202054e-08,7.351677e-07,3.037910e-07,6.441360e-07,1.755551e-06,...,1.894127e-06,6.333711e-07,1.163913e-05,9.687103e-08,Blank,Human,Elephant_African,0.992329,0.006903,0.000619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43623,1.663137e-03,0.001725,3.458983e-05,1.380556e-04,0.000029,1.352274e-03,6.608617e-05,5.736975e-03,1.284553e-02,1.925576e-04,...,3.190487e-05,4.596190e-05,2.554128e-05,1.658706e-03,Elephant_African,Duiker_Red,Duiker_Blue,0.965145,0.012846,0.005737
43624,4.539652e-05,0.000130,9.310187e-07,6.942720e-06,0.000002,4.915390e-04,3.323348e-06,7.082932e-07,9.804475e-06,2.017840e-05,...,1.897237e-05,1.782077e-07,1.560462e-06,2.459209e-07,Leopard_African,Chimpanzee,Human,0.998709,0.000492,0.000337
43625,1.762118e-04,0.004880,5.249129e-06,1.339516e-04,0.000060,5.211334e-01,5.145297e-05,4.796041e-04,7.022453e-05,4.422554e-06,...,4.142888e-05,3.136026e-05,9.774395e-06,4.366212e-04,Chimpanzee,Gorilla,Elephant_African,0.521133,0.420494,0.026563
43626,3.938063e-06,0.000032,4.535523e-08,5.333808e-07,0.000001,4.047143e-07,6.454020e-07,4.682421e-06,6.077072e-06,3.763003e-07,...,5.094130e-07,2.151043e-06,3.929028e-07,9.069347e-06,Elephant_African,Human,Blank,0.998539,0.001377,0.000032


In [63]:
df = df.reset_index()

In [68]:
correct_p = (df.species == df_preds.pred_1).sum()
all_p = len(df)

In [69]:
print(f"accuracy is: {round(100 * correct_p / all_p,2)}%")

accuracy is: 75.98%


So accuracy is slightly lower than on the bigger images. This is worth keeping in mind.